In [ ]:
#Revised September 2019 - remove plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
#from mpl_toolkits.basemap import Basemap
import datetime
from scipy import signal

import matplotlib.dates as mdates

In [ ]:
%matplotlib inline
plt.ion()

In [ ]:
#full list
log_files=[['/home/pkotze/python/SKA-MPI Power measurements/power_test_A1_1/CSV/97intvl.csv','Initial power check']]


In [ ]:
for log_file,description in log_files:   #print(log_file)
    df=pd.read_csv(log_file,header=1,parse_dates=['Time'])
    #sub_range=list(range(3,df.shape[0])) #getting number of entries?  could go less
        #print(type(df))
    #overview(df,log_file + ' * ' + description)
    #OAT(df,log_file + ' * ' + description)
    #fuel(df,log_file + ' * ' + description)
    #airspeed(df,log_file + ' * ' + description)
    
    

In [ ]:
df.tail()



In [ ]:
df.columns


In [ ]:
timestamps=(df['Time'])

In [ ]:
type(timestamps[0])


Check timestamps step size to see if data average/interval was changed.

In [ ]:
timestamps

In [ ]:
#1 - Instantaneous Inrush [VA]
#line to neutral voltage
maxurms1=np.asarray( df['MaxUrms1'] ).astype(float)
maxurms2=np.asarray( df['MaxUrms2'] ).astype(float)
maxurms3=np.asarray( df['MaxUrms3'] ).astype(float)
maxirms1=np.asarray( df['MaxIrms1'] ).astype(float)
maxirms2=np.asarray( df['MaxIrms2'] ).astype(float)
maxirms3=np.asarray( df['MaxIrms3'] ).astype(float)

maxs1=maxurms1*maxirms1
maxs2=maxurms2*maxirms2
maxs3=maxurms2*maxirms3

#Average P Q S power
maxpsum=np.asarray( df['MaxPsum'] ).astype(float) #active?
maxqsum=np.asarray( df['MaxQsum'] ).astype(float) #reactive?
maxssum=np.asarray( df['MaxSsum'] ).astype(float) #total?

#2 - Short term < 5s [W]
#3 - Long term > 5s [W]

#average PF sum
avepfsum=np.asarray( df['AvePFsum'] ).astype(float)

#frequency 
minfreq=np.asarray( df['MinFreq'] ).astype(float)
avefreq=np.asarray( df['AveFreq'] ).astype(float)
maxfreq=np.asarray( df['MaxFreq'] ).astype(float)

#average  rms line currents?
aveirms1=np.asarray( df['AveIrms1'] ).astype(float)
aveirms2=np.asarray( df['AveIrms2'] ).astype(float)
aveirms3=np.asarray( df['AveIrms3'] ).astype(float)
aveirms4=np.asarray( df['AveIrms4'] ).astype(float)

#Average P Q S power
avepsum=np.asarray( df['AvePsum'] ).astype(float) #active?
aveqsum=np.asarray( df['AveQsum'] ).astype(float) #reactive?
avessum=np.asarray( df['AveSsum'] ).astype(float) #total?


In [ ]:

#Plot calculated peak 
fig,ax = plt.subplots(figsize=(20,15))

ax.set(title="Basic hioki parameters: =>  "+log_file, ylabel='Apparent power [VA]', 
xlabel='Time [s]')
ax.plot(timestamps,maxs1)
ax.plot(timestamps,maxs2)
ax.plot(timestamps,maxs3)
ax.plot(timestamps,maxpsum)
ax.plot(timestamps,maxqsum)
ax.plot(timestamps,maxssum)

ymin=-3000
ymax=3000
ystep=100
plt.yticks(range(ymin,ymax,ystep))
plt.ylim(ymin,ymax)
ax.xaxis.set_major_locator(mdates.SecondLocator(interval=10))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S')) 
_=plt.xticks(rotation=65)
ax.legend(['Phase 1 max S [VA]','Phase 2 max S [VA]','Phase 3 max S [VA]','p','q','s'])
ax.grid()


In [ ]:

#Plot Basic P Q S power parameters

fig,ax = plt.subplots(figsize=(20,15))

ax.set(title="Basic hioki parameters: =>  "+log_file, ylabel='AvePMaxSsum [W?]', 
xlabel='Time [s]')
ax.plot(timestamps,avepsum)
ax.plot(timestamps,aveqsum)
ax.plot(timestamps,avessum)


ymin=-3000
ymax=3000
ystep=100
plt.yticks(range(ymin,ymax,ystep))
plt.ylim(ymin,ymax)
ax.xaxis.set_major_locator(mdates.SecondLocator(interval=10))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S')) 
_=plt.xticks(rotation=65)
ax.legend(['P [W]','Q [VAR]','S [VA]'])
ax.grid()


In [ ]:
#line to neutral voltage
aveirms1=np.asarray( df['AveUrms1'] ).astype(float)
aveirms2=np.asarray( df['AveUrms2'] ).astype(float)
aveirms3=np.asarray( df['AveUrms3'] ).astype(float)

#average PF sum
avepfsum=np.asarray( df['AvePFsum'] ).astype(float)

#frequency 
minfreq=np.asarray( df['MinFreq'] ).astype(float)
avefreq=np.asarray( df['AveFreq'] ).astype(float)
maxfreq=np.asarray( df['MaxFreq'] ).astype(float)

#average  rms line currents?
aveirms1=np.asarray( df['AveIrms1'] ).astype(float)
aveirms2=np.asarray( df['AveIrms2'] ).astype(float)
aveirms3=np.asarray( df['AveIrms3'] ).astype(float)
aveirms4=np.asarray( df['AveIrms4'] ).astype(float)

#Average P Q S power
avepsum=np.asarray( df['AvePsum'] ).astype(float) #active?
aveqsum=np.asarray( df['AveQsum'] ).astype(float) #reactive?
avessum=np.asarray( df['AveSsum'] ).astype(float) #total?


In [ ]:


fig,ax = plt.subplots(figsize=(20,15))

ax.set(title="Basic hioki parameters: =>  "+log_file, ylabel='AvePMaxSsum [W?]', 
xlabel='Time [s]')
ax.plot(timestamps,avepsum)
ax.plot(timestamps,aveqsum)
ax.plot(timestamps,avessum)


ymin=0
ymax=3000
ystep=100
#plt.yticks(range(ymin,ymax,ystep))
#plt.ylim(ymin,ymax)
ax.xaxis.set_major_locator(mdates.SecondLocator(interval=10))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S')) 
_=plt.xticks(rotation=65)
ax.legend(['P','Q','S'])
ax.grid()


In [ ]:
#Plot Basic engine parameters

fig,ax = plt.subplots(figsize=(20,20))

ax.set(title="Basic hioki parameters: =>  "+log_file, ylabel='Ave I RMS', 
xlabel='Time [s]')
ax.plot(timestamps,aveirms1)
ax.plot(timestamps,aveirms2)
ax.plot(timestamps,aveirms3)
ax.plot(timestamps,aveirms4)

ax.xaxis.set_major_locator(mdates.SecondLocator(interval=10))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S')) 
_=plt.xticks(rotation=90)
ax.legend(['1','2','3','4'])
ax.grid()


In [ ]:
#Plot Basic engine parameters

fig,ax = plt.subplots(figsize=(20,20))

ax.set(title="Basic hioki parameters: =>  "+log_file, ylabel='Ave I RMS', 
xlabel='Time [s]')
ax.plot(timestamps,avepfsum)

ax.xaxis.set_major_locator(mdates.SecondLocator(interval=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S')) 

ax.legend(['1','2','3','4'])
ax.grid()

In [ ]:

fig,ax = plt.subplots(figsize=(20,20))

ax.set(title="Basic hioki parameters: =>  "+log_file, ylabel='AvePMaxSsum [W?]', 
xlabel='Time [s]')
ax.plot(timestamps,minfreq)
ax.plot(timestamps,avefreq)
ax.plot(timestamps,maxfreq)


ymin=0
ymax=3000
ystep=100
#plt.yticks(range(ymin,ymax,ystep))
#plt.ylim(ymin,ymax)
ax.xaxis.set_major_locator(mdates.SecondLocator(interval=30))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S')) 
ax.legend(['Min','aVe','Max'])
ax.grid()
